# LangChain 1.0 Agent with MCP Server Functions

## Overview
This notebook demonstrates how to integrate **Model Context Protocol (MCP)** servers with **LangChain 1.0** agents.

**What you'll learn:**
- Converting MCP server functions into LangChain tools
- Creating agents that use MCP-exported functions
- Single agent pattern (direct MCP tool usage)
- Multi-agent supervisor pattern (coordinated MCP usage)

**References:**
- MCP Servers: https://github.com/modelcontextprotocol/servers
- LangChain 1.0: https://python.langchain.com/docs/modules/agents/
- Multi-Agent Pattern: https://github.com/cloudyuga/agentic-ai-w-mcp

## Setup and Installation

In [ ]:
# Install required packages
!pip install -q langchain langchain-openai langchain-core pytz python-dotenv

In [ ]:
# Import libraries
import os
import json
from datetime import datetime
import pytz
from dotenv import load_dotenv

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.tools import tool
from langchain.agents import create_agent

# Load environment variables
load_dotenv()

print("✓ Libraries imported successfully!")

In [ ]:
# Set your OpenAI API key
# Option 1: Set in environment
# os.environ["OPENAI_API_KEY"] = "your-key-here"

# Option 2: Load from .env file (recommended)
# Create a .env file with: OPENAI_API_KEY=your-key-here

# Verify API key is set
if not os.getenv("OPENAI_API_KEY"):
    print("⚠️ Warning: OPENAI_API_KEY not set")
    print("Please set it with: os.environ['OPENAI_API_KEY'] = 'your-key-here'")
else:
    print("✓ OPENAI_API_KEY is configured")

## Part 1: Mock MCP Server (Time Server)

For this example, we'll use a **mock MCP server** that simulates the functionality of the official `time` MCP server.

In production, you would connect to a real MCP server, but this makes the example self-contained and easy to run.

In [ ]:
class MockMCPTimeServer:
    """
    Mock MCP server that simulates the 'time' MCP server functionality.
    
    In production, this would be replaced with actual MCP ClientSession.
    """
    
    @staticmethod
    def get_current_time(timezone: str = "UTC") -> str:
        """Get current time in specified timezone"""
        try:
            tz = pytz.timezone(timezone)
            current_time = datetime.now(tz)
            return current_time.strftime("%Y-%m-%d %H:%M:%S %Z")
        except Exception as e:
            return f"Error: {str(e)}"
    
    @staticmethod
    def list_timezones() -> str:
        """List common timezones"""
        common_zones = [
            "UTC", "America/New_York", "America/Los_Angeles",
            "Europe/London", "Europe/Paris", "Asia/Tokyo",
            "Asia/Shanghai", "Australia/Sydney", "Asia/Kolkata"
        ]
        return json.dumps(common_zones, indent=2)
    
    @staticmethod
    def convert_time(time_str: str, from_tz: str, to_tz: str) -> str:
        """Convert time between timezones"""
        try:
            # Parse the time string (assuming format: "HH:MM")
            time_parts = time_str.split(":")
            hour = int(time_parts[0])
            minute = int(time_parts[1]) if len(time_parts) > 1 else 0
            
            # Create datetime in source timezone
            from_timezone = pytz.timezone(from_tz)
            to_timezone = pytz.timezone(to_tz)
            
            # Use today's date with the specified time
            now = datetime.now()
            dt = from_timezone.localize(datetime(now.year, now.month, now.day, hour, minute))
            
            # Convert to target timezone
            converted = dt.astimezone(to_timezone)
            
            return converted.strftime("%Y-%m-%d %H:%M:%S %Z")
        except Exception as e:
            return f"Error: {str(e)}"

# Initialize mock MCP server
mock_mcp_server = MockMCPTimeServer()

# Test the mock server
print("Mock MCP Server initialized!")
print(f"Test: Current time in UTC: {mock_mcp_server.get_current_time('UTC')}")

## Part 2: Convert MCP Functions to LangChain Tools

This is the **KEY STEP**: We wrap each MCP server function as a LangChain `@tool` decorator.

This allows LangChain agents to discover and use these functions.

In [ ]:
@tool
def get_current_time(timezone: str = "UTC") -> str:
    """Get the current time in a specific timezone.
    
    Args:
        timezone: IANA timezone name (e.g., 'America/New_York', 'Asia/Tokyo', 'UTC')
        
    Returns:
        Current time in the specified timezone
        
    Example:
        get_current_time("Asia/Tokyo") -> "2024-01-15 14:30:00 JST"
    """
    return mock_mcp_server.get_current_time(timezone)


@tool
def list_timezones() -> str:
    """List common timezone names that can be used with time functions.
    
    Returns:
        JSON array of common timezone names
    """
    return mock_mcp_server.list_timezones()


@tool
def convert_time(time_str: str, from_tz: str, to_tz: str) -> str:
    """Convert a time from one timezone to another.
    
    Args:
        time_str: Time to convert in HH:MM format (e.g., "14:30")
        from_tz: Source timezone (e.g., "America/New_York")
        to_tz: Target timezone (e.g., "Asia/Tokyo")
        
    Returns:
        Converted time with date in target timezone
        
    Example:
        convert_time("14:30", "America/New_York", "Asia/Tokyo")
    """
    return mock_mcp_server.convert_time(time_str, from_tz, to_tz)


# Collection of all MCP-derived tools
mcp_tools = [get_current_time, list_timezones, convert_time]

print("✓ MCP tools created:")
for t in mcp_tools:
    print(f"  - {t.name}")

## Part 3: Single Agent Pattern

Create a **single agent** that has direct access to all MCP tools.

This pattern is good for:
- Simple tasks
- Single domain (e.g., only time operations)
- Few tools (< 10)

In [ ]:
def create_time_assistant():
    """Create a time assistant agent using MCP tools"""
    
    agent = create_agent(
        model="openai:gpt-4o-mini",  # LangChain 1.0 model format
        tools=mcp_tools,  # MCP-derived tools
        system_prompt="""You are a helpful time assistant powered by MCP (Model Context Protocol) server functions.

Your capabilities come from MCP server tools:
- get_current_time: Get current time in any timezone
- list_timezones: Show available timezones
- convert_time: Convert times between timezones

Always:
1. Use the appropriate MCP tool for each request
2. Provide clear, formatted responses
3. Include timezone names in your answers
4. Be helpful and accurate"""
    )
    
    return agent


# Create the agent
time_agent = create_time_assistant()

print("✓ Single agent created with MCP tools!")

### Test Single Agent

In [ ]:
# Test Query 1: Simple time query
query = "What time is it in Tokyo right now?"

print(f"📝 Query: {query}")
print("-" * 80)

result = time_agent.invoke({"messages": [HumanMessage(content=query)]})

# Show tool calls
print("\n🔧 MCP Tools Called:")
for msg in result['messages']:
    if isinstance(msg, AIMessage) and hasattr(msg, 'tool_calls') and msg.tool_calls:
        for tc in msg.tool_calls:
            print(f"  ✓ {tc['name']}")

# Show response
print("\n💬 Agent Response:")
for msg in reversed(result['messages']):
    if isinstance(msg, AIMessage) and msg.content:
        if not hasattr(msg, 'tool_calls') or not msg.tool_calls:
            print(f"  {msg.content}")
            break

In [ ]:
# Test Query 2: Time conversion
query = "Convert 2:30 PM from America/New_York to Europe/London"

print(f"📝 Query: {query}")
print("-" * 80)

result = time_agent.invoke({"messages": [HumanMessage(content=query)]})

# Show tool calls
print("\n🔧 MCP Tools Called:")
for msg in result['messages']:
    if isinstance(msg, AIMessage) and hasattr(msg, 'tool_calls') and msg.tool_calls:
        for tc in msg.tool_calls:
            print(f"  ✓ {tc['name']}")

# Show response
print("\n💬 Agent Response:")
for msg in reversed(result['messages']):
    if isinstance(msg, AIMessage) and msg.content:
        if not hasattr(msg, 'tool_calls') or not msg.tool_calls:
            print(f"  {msg.content}")
            break

## Part 4: Multi-Agent Supervisor Pattern

Create a **supervisor pattern** with specialized agents:
- **Supervisor**: Routes requests to appropriate specialists
- **Time Query Specialist**: Handles "what time is it" queries
- **Time Conversion Specialist**: Handles timezone conversions

This pattern is good for:
- Complex tasks
- Multiple domains
- Many tools (10+)
- Need for specialization

In [ ]:
# Create specialized sub-agents

# Sub-Agent 1: Time Query Specialist
time_query_specialist_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_current_time, list_timezones],
    system_prompt="""You are a Time Query Specialist.

You handle questions about current time in different locations.
Use get_current_time and list_timezones MCP tools.

You ONLY answer "what time is it" questions.
You do NOT handle time conversions - that's another specialist's job."""
)

# Sub-Agent 2: Time Conversion Specialist
time_conversion_specialist_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[convert_time, list_timezones],
    system_prompt="""You are a Time Conversion Specialist.

You convert times between different timezones.
Use convert_time and list_timezones MCP tools.

You ONLY handle conversion requests.
You do NOT answer simple "what time is it" queries."""
)

print("✓ Specialist agents created!")

In [ ]:
# Wrap specialists as tools for the supervisor

@tool
def time_query_specialist(request: str) -> str:
    """Handle queries about current time in different locations.
    
    Use this when the user asks:
    - "What time is it in [location]?"
    - "Show me current time in [timezone]"
    - "What are the available timezones?"
    
    Args:
        request: User's time query
        
    Returns:
        Response with current time information
    """
    result = time_query_specialist_agent.invoke(
        {"messages": [HumanMessage(content=request)]}
    )
    
    # Extract final response
    for msg in reversed(result['messages']):
        if isinstance(msg, AIMessage) and msg.content:
            if not hasattr(msg, 'tool_calls') or not msg.tool_calls:
                return msg.content
    
    return "Time query processed."


@tool
def time_conversion_specialist(request: str) -> str:
    """Handle time conversion between timezones.
    
    Use this when the user asks to:
    - Convert a time from one timezone to another
    - Calculate time differences
    - Schedule meetings across timezones
    
    Args:
        request: User's conversion request
        
    Returns:
        Response with converted time
    """
    result = time_conversion_specialist_agent.invoke(
        {"messages": [HumanMessage(content=request)]}
    )
    
    for msg in reversed(result['messages']):
        if isinstance(msg, AIMessage) and msg.content:
            if not hasattr(msg, 'tool_calls') or not msg.tool_calls:
                return msg.content
    
    return "Time conversion processed."


print("✓ Specialist tools wrapped!")

In [ ]:
# Create the supervisor agent

supervisor_tools = [time_query_specialist, time_conversion_specialist]

supervisor_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=supervisor_tools,
    system_prompt="""You are a Time Services Supervisor coordinating specialized MCP-powered agents.

Available Specialists (each using MCP server functions):

1. time_query_specialist
   - Handles: "What time is it?" queries
   - Uses MCP tools: get_current_time, list_timezones

2. time_conversion_specialist  
   - Handles: Time zone conversions
   - Uses MCP tools: convert_time, list_timezones

Your Job:
- Analyze the user's request
- Route to the appropriate specialist
- Synthesize their response into a clear answer

Decision Rules:
- "What time is it..." → time_query_specialist
- "Convert..." or "What time will..." → time_conversion_specialist
- Complex requests → use both specialists in sequence"""
)

print("✓ Supervisor agent created!")

### Test Supervisor Pattern

In [ ]:
# Test 1: Simple time query (should route to time_query_specialist)
query = "What time is it in Sydney?"

print(f"📝 Query: {query}")
print("-" * 80)

result = supervisor_agent.invoke({"messages": [HumanMessage(content=query)]})

# Show routing decisions
print("\n🎯 Supervisor Routing:")
for msg in result['messages']:
    if isinstance(msg, AIMessage) and hasattr(msg, 'tool_calls') and msg.tool_calls:
        for tc in msg.tool_calls:
            print(f"  ✓ Routed to: {tc['name']}")

# Show final response
print("\n💬 Final Response:")
for msg in reversed(result['messages']):
    if isinstance(msg, AIMessage) and msg.content:
        if not hasattr(msg, 'tool_calls') or not msg.tool_calls:
            print(f"  {msg.content}")
            break

In [ ]:
# Test 2: Time conversion (should route to time_conversion_specialist)
query = "I have a meeting at 3 PM EST, what time is that in Tokyo?"

print(f"📝 Query: {query}")
print("-" * 80)

result = supervisor_agent.invoke({"messages": [HumanMessage(content=query)]})

# Show routing decisions
print("\n🎯 Supervisor Routing:")
for msg in result['messages']:
    if isinstance(msg, AIMessage) and hasattr(msg, 'tool_calls') and msg.tool_calls:
        for tc in msg.tool_calls:
            print(f"  ✓ Routed to: {tc['name']}")

# Show final response
print("\n💬 Final Response:")
for msg in reversed(result['messages']):
    if isinstance(msg, AIMessage) and msg.content:
        if not hasattr(msg, 'tool_calls') or not msg.tool_calls:
            print(f"  {msg.content}")
            break

## Summary

### What We Demonstrated

1. **MCP Server Integration**
   - Simulated an MCP time server
   - Exposed functions: get_current_time, list_timezones, convert_time

2. **Tool Conversion**
   - Wrapped MCP functions as LangChain `@tool` decorators
   - Made them discoverable by LangChain agents

3. **Single Agent Pattern**
   - Created agent with direct access to all MCP tools
   - Agent autonomously selects and uses appropriate tools
   - Good for simple, single-domain tasks

4. **Multi-Agent Supervisor Pattern**
   - Created specialized sub-agents (query vs conversion)
   - Each specialist has focused MCP tools
   - Supervisor routes requests to appropriate specialist
   - Good for complex, multi-domain tasks

### Key Concepts

```
MCP Server Functions → LangChain Tools (@tool) → Agent (uses tools)
```

### Next Steps

1. **Try Different Queries**: Modify the test cells with your own questions
2. **Add New MCP Functions**: Extend the mock server with more capabilities
3. **Connect Real MCP Server**: Replace mock with actual MCP ClientSession
4. **Build Your Own**: Apply this pattern to your use case (database, API, files, etc.)

### Resources

- **MCP Servers**: https://github.com/modelcontextprotocol/servers
- **LangChain Docs**: https://python.langchain.com/docs/
- **Example Code**: See the .py files in this package